<a href="https://colab.research.google.com/github/yanglilizhang/Face-Recognition/blob/main/yolov8_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
fareselmenshawii_face_detection_dataset_path = kagglehub.dataset_download('fareselmenshawii/face-detection-dataset')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import shutil
import cv2
import random


In [ ]:
import torch

In [ ]:
cwd = os.getcwd()

# Preparing data folder structure

In [ ]:
dirs = [
    "dataset/train/images",
    "dataset/train/labels",
    "dataset/val/images",
    "dataset/val/labels",
    "dataset/test/images",
    "dataset/test/labels",
]

for dir_path in dirs:
    path = os.path.join(cwd, dir_path)
    os.makedirs(path, exist_ok=True)
    print(f"Created: {path}")

In [ ]:
data_path = "/kaggle/input/face-detection-dataset"

Copy train images and label

In [ ]:
train_imgs_src = os.path.join(data_path,"images","train")
train_labels_src = os.path.join(data_path,"labels","train")

dest = os.path.join(cwd,"dataset","train")

train_imgs_list = sorted(os.listdir(train_imgs_src))
train_labels_list = sorted(os.listdir(train_labels_src))

total = len(train_imgs_list)
i=1
for img,label in zip(train_imgs_list,train_labels_list):
    src_img = os.path.join(train_imgs_src,img)
    img_path =  os.path.join(dest,"images",img)

    src_label = os.path.join(train_labels_src,label)
    label_path =  os.path.join(dest,"labels",label)

    print(f'copying {src_img} to {img_path}')
    shutil.copy(src_img,img_path)
    print(f'copying {src_label} to {label_path}')
    shutil.copy(src_label,label_path)

    print(f"Copied {i} images.")
    i+=1

In [ ]:
len(os.listdir(os.path.join(dest,"images"))),len(os.listdir(os.path.join(dest,"labels")))

Split val into val and test 50-50

In [ ]:
val_path_imgs = sorted(os.listdir(os.path.join(data_path,"images","val")))
print(val_path_imgs[:5])
val_path_labels = sorted(os.listdir(os.path.join(data_path,"labels","val")))
print(val_path_labels[:5])

In [ ]:
valimgsList = val_path_imgs[:round(len(val_path_imgs)/2)]
testimgsList = val_path_imgs[round(len(val_path_imgs)/2):]

vallabelsList = val_path_labels[:round(len(val_path_labels)/2)]
testlabelsList = val_path_labels[round(len(val_path_labels)/2):]

len(valimgsList),len(testimgsList),len(vallabelsList),len(testlabelsList)

In [ ]:
imgs_src = os.path.join(data_path,"images","val")
labels_src = os.path.join(data_path,"labels","val")

dest_val = os.path.join(cwd,"dataset","val")
dest_test = os.path.join(cwd,"dataset","test")

def move_val_test(dest,imgs_list,labels_list):
    total = len(imgs_list)
    i=1
    for img,label in zip(imgs_list,labels_list):
        src_img = os.path.join(imgs_src,img)
        img_path =  os.path.join(dest,"images",img)

        src_label = os.path.join(labels_src,label)
        label_path =  os.path.join(dest,"labels",label)

        print(f'copying {src_img} to {img_path}')
        shutil.copy(src_img,img_path)
        print(f'copying {src_label} to {label_path}')
        shutil.copy(src_label,label_path)

        print(f"Copied {i} images.")
        i+=1

In [ ]:
# copy to val
move_val_test(dest_val,valimgsList,vallabelsList)

# copy to test
move_val_test(dest_test,testimgsList,testlabelsList)

len(os.listdir(os.path.join(dest_test,"images"))),len(os.listdir(os.path.join(dest_test,"labels"))), len(os.listdir(os.path.join(dest_val,"images"))),len(os.listdir(os.path.join(dest_val,"labels"))),


In [ ]:
len(os.listdir(os.path.join(dest_test,"images"))),len(os.listdir(os.path.join(dest_test,"labels"))), len(os.listdir(os.path.join(dest_val,"images"))),len(os.listdir(os.path.join(dest_val,"labels"))),


# Prepare config file

In [ ]:
ds = os.path.join(cwd,"dataset")
ln_1="#Paths \n"
ln_2='train: ' +"'"+os.path.join(ds,"train","images")+"' \n"
ln_3='val: ' +"'" + os.path.join(ds,"val","images")+"' \n"
ln_4='test: ' +"'" + os.path.join(ds,"test","images")+"' \n"
ln_5="\n"
ln_6='# Classes\n'
ln_7='names:\n'
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

In [ ]:
with open("config.yaml", 'w') as f:
    f.writelines(config_lines)

In [ ]:
config_path = os.path.join(cwd,"config.yaml")

In [ ]:
!pip install ultralytics

# Training

In [ ]:
from ultralytics import YOLO

In [ ]:
# Using YOLO's ptetrained model architecture and weights for training
model=YOLO('yolov8s.yaml').load('yolov8s.pt')

In [ ]:
model.train(
    data= config_path,
    epochs=50,
    batch=32,
    workers=4,
    resume=True,
    device=0,
    name='model_1',  # Custom experiment name
    save=True               # Ensure saving is enabled
)


In [ ]:
def display_curves(root_path):
    plt.figure(figsize=(50,50))

    #displaying p curve
    p_curve=cv2.imread(os.path.join(root_path,'P_curve.png'))
    ax=plt.subplot(5,1,1)
    plt.imshow(p_curve)

    #displaying r curve
    r_curve=cv2.imread(os.path.join(root_path,'R_curve.png'))
    ax=plt.subplot(5,1,2)
    plt.imshow(r_curve)

    #displaying pr curve
    pr_curve=cv2.imread(os.path.join(root_path,'PR_curve.png'))
    ax=plt.subplot(5,1,3)
    plt.imshow(pr_curve)

    #displaying f1 curve
    f1_curve=cv2.imread(os.path.join(root_path,'F1_curve.png'))
    ax=plt.subplot(5,1,4)
    plt.imshow(f1_curve)

    #displaying confusion matrix
    confusion_matrix=cv2.imread(os.path.join(root_path,'confusion_matrix.png'))
    ax=plt.subplot(5,1,5)
    plt.imshow(confusion_matrix)

# Evaluate model

In [ ]:
model_1 = YOLO("/kaggle/working/runs/detect/model_12/weights/best.pt")
model_1.info()

In [ ]:
def evaluate_map50(trainedmodel, data_path, dataset='val', **kwargs):
    metrics=trainedmodel.val(data=data_path, split=dataset, **kwargs)
    map50=round(metrics.box.map50, 3)
    print("The mAP of model on {0} dataset is {1}".format(dataset,map50))
    return metrics, map50

In [ ]:
# metrics,map50_val = evaluate_map50(model_1, os.path.join(cwd,"dataset"),dataset='val' )
metrics,map50_val = evaluate_map50(model_1, config_path,dataset='val' )


In [ ]:
metrics,map50_test2 = evaluate_map50(model_1, config_path,dataset='test',device="cuda:0",plots=True,conf=0.6,iou=0.4)

In [ ]:
path = "/kaggle/working/runs/detect/val8"
display_curves(path)

In [ ]:
metrics,map50_test2 = evaluate_map50(model_1, config_path,dataset='test',device="cuda:0",plots=True,conf=0.3,iou=0.8)


In [ ]:
path = "/kaggle/working/runs/detect/val12"
display_curves(path)

# Plot random images and their results

sample a few random images and plot their results

In [ ]:
imgtestpath = "/kaggle/working/dataset/test/images"

n = 10
test_images = random.sample(os.listdir(imgtestpath), n)

plt.figure(figsize=(15, 30))

for idx, img_name in enumerate(test_images):
    # Full path to the test image
    test_image_path = os.path.join(imgtestpath, img_name)

    # Read and display the actual image
    ax = plt.subplot(n, 2, 2 * idx + 1)
    image = cv2.imread(test_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title("Actual Image", fontsize=20)

    # Predict using the YOLO model
    res = model_1.predict(test_image_path, iou=0.8, conf=0.3)
    res_plotted = res[0].plot()  # Get the annotated image from YOLO

    # Display image with predictions
    ax = plt.subplot(n, 2, 2 * idx + 2)
    plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Image with Predictions", fontsize=20)

# Show the plots
plt.tight_layout()
plt.show()


sample a random image and get the cropped version

In [ ]:
# random.seed(69)
test_image = os.listdir(imgtestpath)[random.randint(0,1000)]
test_image_path = os.path.join(imgtestpath, test_image)
image = cv2.imread(test_image_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')

In [ ]:
results = model_1.predict(test_image_path, iou=0.8, conf=0.3)
# Assuming `results` is from your YOLO model inference
bounding_boxes = results[0].boxes.xyxy  # Tensor of shape [num_boxes, 4]

# Load the image
image = cv2.imread(test_image_path)  # BGR format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

# Loop through bounding boxes and crop
for idx, box in enumerate(bounding_boxes):
    # Convert tensor coordinates to integers
    x_min, y_min, x_max, y_max = map(int, box.tolist())

    # Crop the image using slicing
    cropped_face = image[y_min:y_max, x_min:x_max]

    # Display the cropped face
    plt.figure(figsize=(5, 5))
    plt.imshow(cropped_face)
    plt.axis('off')
    plt.title(f"Cropped Face {idx + 1}")
    plt.show()

    # Optionally save the cropped face
    save_path = f"cropped_face_{idx + 1}.jpg"
    cv2.imwrite(save_path, cv2.cvtColor(cropped_face, cv2.COLOR_RGB2BGR))


In [ ]:
model.save(os.path.join(cwd,"face_detection.pt"))

In [ ]:
# function to remove any file to clean up the outputs
def remove_path(path):
    """Remove a file or directory (empty or non-empty)."""
    if os.path.isfile(path):
        os.remove(path)  # Remove file
        print(f"File {path} has been removed")
    elif os.path.isdir(path):
        try:
            os.rmdir(path)  # Try to remove empty directory
            print(f"Empty directory {path} has been removed")
        except OSError:
            shutil.rmtree(path)  # Remove non-empty directory
            print(f"Non-empty directory {path} and its contents have been removed")
    else:
        print(f"{path} does not exist or is not a valid file or directory")
